# Automatisation de l'Importation - Notebook

## I - Importation des modules

In [22]:
import json
from shapely import geometry
import pandas as pd
import geopandas

## II - Création des fonctions d'importation

Lecteur:
Prend en entrée le chemin d'accès à un fichier Json, en le convertie en dictionnaire python.

In [3]:
def lecteur(path):
    ''' Ouvre un fichier json en dictionnaire'''
    with open(path, 'r',encoding="utf-8") as file:
        gsv = json.load(file)
    file.close
    return(gsv)

Deplieur_1:
Prend en entrée un dictionnaire Python issue d'un Json, et le "déplie". Dans tous les fichiers Json rencontrés, les données exploitables étaient contenues dans la clé "features" du dictionnaire. On renvoie ici seulement les éléments de la clé "features", si celle-ci existe, et on ne change rien si cette clé n'éxiste pas.

In [5]:
def deplieur_1(gsv):
    '''Déplie la première étape'''
    is_features = False
    for cle, valeur in gsv.items():
        if cle == 'features':
            is_features = True
    if is_features:
        return(gsv['features'])
    else:
        return(gsv)

Deplieur_2: Prend en entrée un dictionnaire une première fois. Souvent, une fois la clé "features", les données sont stockées dans les clés "properties" et "geometry". On vient donc, si ces clés existent, déplier encore une fois les données.

In [6]:
def deplieur_2(gsv):
    '''Déplie la deuxième étape'''
    #On regarde si dans le schéma il y a properties et geometry
    is_prop = False
    is_geom = False
    for cle, valeur in gsv[0].items():
        if cle == 'properties':
            is_prop = True
        if cle == 'geometry':
            is_geom = True
    
    if is_prop and is_geom:
        tab = []
        i=0
        for row in gsv:
            i=i+1
            #print(i)
            is_prop = False
            is_geom = False
            for cle, valeur in row.items():
                if cle == 'properties':
                    is_prop = True
                if cle == 'geometry':
                    is_geom = True
                    if row['geometry'] == None:
                        is_geom = False
            if is_prop and is_geom:
                dic = {}
                for cle, valeur in row['properties'].items():
                    dic[cle] = valeur
                    type_geom = row['geometry']['type']
                    if type_geom == 'Point':
                        geom = geometry.Point(row['geometry']['coordinates'])
                    elif type_geom == 'Polygon':
                        geom = geometry.Polygon(row['geometry']['coordinates'][0])
                    elif type_geom == 'MultiPolygon':
                        geom = geometry.Polygon(row['geometry']['coordinates'][0][0])
                    dic['geometry'] = geom
                tab.append(dic)
        return(tab)

## III - Création des fonctions de transformation

#### /!\ Ces fonctions doivent être adaptées aux données considérées /!\

Arbre_to_gdf: /!\ Ces fonctions sont amenées à être modifiées en fonction des éléments compris dans les fichiers Json traités.
Cette fonction prend en entrée un dictionnaire contenant les données des arbres, et renvoie un GeoDataFrame comprenant les données nécessaires au calcul des ombres : ID, hauteur, rayon, et géométrie.

In [23]:
def arbre_to_gdf(gsv):
    '''Transorme un dictionnaire en GeoDataFrame'''
    tab = []
    for i in range(0, len(gsv)):
        geom = gsv[i]['geometry']
        rayon = gsv[i]['rayoncouronne_m']
        hauteurtotale = gsv[i]['hauteurtotale_m']
        tab.append({'index': i,'hauteur': hauteurtotale,'rayon': rayon, 'geometry': geom})
    df = pd.DataFrame(tab)
    gdf = geopandas.GeoDataFrame(df, crs=4171)
    gdf = gdf.to_crs(2154)
    return(gdf)

Building_to_gdf: /!\ Ces fonctions sont amenées à être modifiées en fonction des éléments compris dans les fichiers Json traités. Cette fonction prend en entrée un dictionnaire contenant les données des buildings, et renvoie un GeoDataFrame comprenant les données nécessaires au calcul des ombres : ID, hauteur et géométrie.

In [24]:
def building_to_gdf(gsv):
    '''Transorme un dictionnaire en GeoDataFrame'''
    tab = []
    for i in range(0, len(gsv)):
        geom = gsv[i]['geometry']
        htotale = gsv[i]['htotale']
        tab.append({'index': i,'hauteur': htotale, 'geometry': geom})
    df = pd.DataFrame(tab)
    gdf = geopandas.GeoDataFrame(df, crs=4171)
    gdf = gdf.to_crs(2154)
    return(gdf)

path_to_gdf: fonction complète. Prend en entrée les 2 chemins d'accès aux fichiers Json des arbres et des bâtiments, et renvoie deux GeoDataFrame formatés.

In [9]:
def path_to_gdf(path_arbre, path_building):
    gsv_arbre = lecteur(path_arbre)
    gsv_arbre = deplieur_1(gsv_arbre)
    gsv_arbre = deplieur_2(gsv_arbre)
    
    gsv_building = lecteur(path_building)
    gsv_building = deplieur_1(gsv_building)
    gsv_building = deplieur_2(gsv_building)
    
    gdf_arbre = arbre_to_gdf(gsv_arbre)
    gdf_building = building_to_gdf(gsv_building)
    return(gdf_arbre, gdf_building)

## IV - Tests

Chemin d'accès aux fichiers Json comprenant les données nécessaires.
[Insérez vos propres chemins d'accès]

In [15]:
path_arbres = "C:/Users/meder/Documents/Etudes/EC Nantes/EI3/P1/Automatisation de l'importation/data_short/arbre_villeurbanne.json"
path_buildings = "C:/Users/meder/Documents/Etudes/EC Nantes/EI3/P1/Automatisation de l'importation/data_short/building_first_100.json"

On applique les fonctions d'importation

In [18]:
[gdf_arbres , gdf_buildings] = path_to_gdf(path_arbres,path_buildings)

In [19]:
gdf_arbres.head()

,index,hauteur,rayon,geometry
0,0,13,4.0,POINT (845278.924 6521633.532)
1,1,9,3.0,POINT (844457.880 6521405.529)
2,2,14,2.5,POINT (845131.459 6521200.974)
3,3,17,6.0,POINT (844928.974 6521661.839)
4,4,6,1.0,POINT (846110.420 6519398.880)


In [20]:
gdf_buildings.head()

,index,hauteur,geometry
0,0,3.4,"POLYGON Z ((855159.722 6513444.392 221.537, 85..."
1,1,2.5,"POLYGON Z ((854945.566 6514323.736 218.425, 85..."
2,2,2.5,"POLYGON Z ((854943.451 6514324.722 218.425, 85..."
3,3,5.3,"POLYGON Z ((854979.496 6514247.819 218.720, 85..."
4,4,9.6,"POLYGON Z ((852354.118 6514406.654 213.114, 85..."
